In [1]:
import sys; sys.path.append("../../../automl/")

In [2]:
from pathlib import Path
import yaml
import joblib

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from src.automl.model.lama import TabularLamaUtilized
from src.automl.loggers import configure_root_logger
from src.automl.constants import create_ml_data_dir
from src.automl.model.metrics import RocAuc

In [3]:
create_ml_data_dir()
configure_root_logger()

## Constants

In [4]:
RANDOM_SEED = 77
DATA_PATH = Path("../../../data/")
CONFIG_PATH = Path("../../../configs/config.yaml")
N_JOBS = 16

In [5]:
with CONFIG_PATH.open() as f:
    cfg = yaml.load(f, Loader=yaml.SafeLoader)

## Data

In [10]:
df_train = pd.read_parquet(DATA_PATH / "train_preproc_2.parquet")
df_train, df_test = df_train.sort_values(by="id").iloc[:300_000], df_train.sort_values(by="id").iloc[300_000:]

In [11]:
df_train["target"].value_counts()

0    282256
1     17744
Name: target, dtype: int64

In [9]:
# undersample the 0 class
#df_train = pd.concat([df_train.loc[df_train.target == 1], df_train.loc[df_train.target == 0].sample(200_000, random_state=RANDOM_SEED)], ignore_index=True)


In [12]:
cat_columns = df_train.drop(columns=["target", "id"]).select_dtypes(int).columns.values.tolist()

In [13]:
X_train, y_train = df_train[cfg["selected_features"] + cat_columns], df_train["target"]
X_test, y_test = df_test[cfg["selected_features"] + cat_columns], df_test["target"]

In [14]:
display(y_train.value_counts(normalize=True))
display(y_test.value_counts(normalize=True))

0    0.940853
1    0.059147
Name: target, dtype: float64

0    0.941322
1    0.058678
Name: target, dtype: float64

In [16]:
#categorical_features = ohe_cols# + oe_cols

## Model

In [16]:
metric = RocAuc()

In [17]:
model = TabularLamaUtilized(n_jobs=16, task="classification")
model.tune(X_train, y_train, metric, timeout=60 * 60, categorical_features=cat_columns)
oof = model.fit(X_train, y_train, categorical_features=cat_columns)
y_pred = model.predict(X_test)

print(metric(y_train, oof))
print(metric(y_test, y_pred))

[2024-11-06 15:11:26,708] - [   START    ] - Fitting TabularLamaUtilized
[15:11:26] Start automl utilizator with listed constraints:
[15:11:26] - time: 7200.00 seconds
[15:11:26] - CPU: 16 cores
[15:11:26] - memory: 16 GB

[15:11:26] If one preset completes earlier, next preset configuration will be started

[15:11:26] ==================================================
[15:11:26] Start 0 automl preset configuration:
[15:11:26] conf_0_sel_type_0.yml, random state: {'reader_params': {'random_state': 42}, 'nn_params': {'random_state': 42}, 'general_params': {'return_all_predictions': False}}
[15:11:26] Stdout logging level is INFO.
[15:11:26] Task: binary

[15:11:26] Start automl preset with listed constraints:
[15:11:26] - time: 7200.00 seconds
[15:11:26] - CPU: 16 cores
[15:11:26] - memory: 16 GB

[15:11:26] Train data shape: (300000, 63)

[15:11:37] Layer 1 train process start. Time left 7189.72 secs
[15:11:45] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[15:12:22] Fitting Lvl_0_Pipe

[15:40:31] Layer 1 train process start. Time left 5455.51 secs
[15:40:40] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[15:41:17] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = 0.7888030003884329
[15:41:17] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[15:41:17] Time left 5408.93 secs

[15:41:25] Selector_LightGBM fitting and predicting completed
[15:41:33] Start fitting Lvl_0_Pipe_1_Mod_0_LightGBM ...
[15:42:11] Fitting Lvl_0_Pipe_1_Mod_0_LightGBM finished. score = 0.8102081895361546
[15:42:11] Lvl_0_Pipe_1_Mod_0_LightGBM fitting and predicting completed
[15:42:11] Start hyperparameters optimization for Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM ... Time budget is 300.00 secs
[15:47:14] Hyperparameters optimization for Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM completed
[15:47:14] Start fitting Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM ...
[15:47:53] Fitting Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM finished. score = 0.8127002275014914
[15:47:53] Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM fitting and p

[16:16:03] Hyperparameters optimization for Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM completed
[16:16:03] Start fitting Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM ...
[16:16:39] Fitting Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM finished. score = 0.812073111942671
[16:16:39] Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM fitting and predicting completed
[16:16:39] Start fitting Lvl_0_Pipe_1_Mod_2_CatBoost ...
[16:17:13] Fitting Lvl_0_Pipe_1_Mod_2_CatBoost finished. score = 0.8072995972551813
[16:17:13] Lvl_0_Pipe_1_Mod_2_CatBoost fitting and predicting completed
[16:17:13] Start hyperparameters optimization for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ... Time budget is 300.00 secs
[16:22:14] Hyperparameters optimization for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost completed
[16:22:14] Start fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ...
[16:23:20] Fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost finished. score = 0.8078172918571538
[16:23:20] Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost fitting and predicting completed
[16:23:20] Time left 2886.61 secs



[16:36:16] Start fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ...
[16:36:48] Fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost finished. score = 0.8065595727647544
[16:36:48] Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost fitting and predicting completed
[16:36:48] Start fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ...
[16:37:40] Fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost finished. score = 0.807202505108077
[16:37:40] Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost fitting and predicting completed
[16:37:40] Hyperparameters optimization for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost completed
[16:37:40] Start fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ...
[16:38:53] Fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost finished. score = 0.8082548081643194
[16:38:53] Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost fitting and predicting completed
[16:38:53] Time left 1953.47 secs

[16:38:53] Layer 1 training completed.

[16:38:53] Blending: optimization starts with equal weights and score 0.8101075398305033
[16:38:57] Blending: iteration 0: score = 0.812783132941

[17:04:39] Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost fitting and predicting completed
[17:04:39] Time left 406.87 secs

[17:04:39] Layer 1 training completed.

[17:04:40] Blending: optimization starts with equal weights and score 0.8105832568389526
[17:04:43] Blending: iteration 0: score = 0.8129764974051146, weights = [0.         0.2100713  0.5476953  0.05638938 0.18584405]
[17:04:47] Blending: iteration 1: score = 0.8130014665044016, weights = [0.         0.25869945 0.5184821  0.         0.22281848]
[17:04:51] Blending: iteration 2: score = 0.8130018483666561, weights = [0.        0.2589566 0.5200349 0.        0.2210085]
[17:04:55] Blending: iteration 3: score = 0.8130018483666561, weights = [0.        0.2589566 0.5200349 0.        0.2210085]
[17:04:55] Blending: no score update. Terminated

[17:04:55] Automl preset training completed in 690.41 seconds

[17:04:55] Model description:
Final prediction for new objects (level 0) = 
	 0.25896 * (5 averaged models Lvl_0_Pipe_1_Mod_0_LightGBM) +
	 

In [19]:
MODEL_NAME = "lamau_8147"
MODEL_DIR = Path(f"../../../data/models/{MODEL_NAME}")
MODEL_DIR.mkdir(exist_ok=True)

In [20]:
res = pd.DataFrame()
res[MODEL_NAME] = oof[:, 1]
res.to_csv(MODEL_DIR / "oof.csv", index=False)
joblib.dump(model, MODEL_DIR / f"{MODEL_NAME}.joblib")

with (MODEL_DIR / "params.yaml").open("w") as f:
    yaml.dump(model.params, f)

with (MODEL_DIR / "score.txt").open("w") as f:
    print("OOF:", metric(y_train, oof), file=f)
    print("Test:", metric(y_test, y_pred), file=f)
    
test = pd.read_parquet(DATA_PATH / "test_preproc_2.parquet")
test["target"] = model.predict(test[cfg["selected_features"] + cat_columns])[:, 1]
test[['id', 'target']].to_csv(MODEL_DIR / f'{MODEL_NAME}.csv', index=False)

## Inference

In [27]:
test = pd.read_parquet(DATA_PATH / "test_preproc_2.parquet")
test["target"] = model.predict(test[cfg["selected_features"] + cat_columns])[:, 1]
test[['id', 'target']].to_csv('lgb_813.csv', index=False)

In [25]:
pred_1 = pd.read_csv("lama_utilized.csv")
pred_2 = pd.read_csv("lgmb_oe_ohe_cols_0805.csv")
pred_3 = pd.read_csv("catboost_ts.csv")

In [26]:
pred_1["target"] = 0.6 * pred_1["target"] + 0.2 * pred_2["target"] + 0.2 * pred_3["target"]

In [29]:
pred_1.to_csv("blend.csv", index=False)

In [166]:
MODEL_DIR.open?

Signature:
MODEL_DIR.open(
    mode='r',
    buffering=-1,
    encoding=None,
    errors=None,
    newline=None,
)
Docstring:
Open the file pointed by this path and return a file object, as
the built-in open() function does.
File:      /usr/lib/python3.10/pathlib.py
Type:      method